In [1]:
import pandas as pd
pd.set_option('display.max_rows' , 200)

In [2]:
company_name = 'TCS'

In [3]:
share_price_in = 1000000;

In [4]:
def calculate_growth(initial , final) :
    if not initial :
        return 1
    return round(((final - initial) / initial) * 100,2)

In [5]:
def get_share_price_in(price) :
    return round(price / share_price_in,2)

    

In [6]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [7]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [8]:
income_statement.rename(columns={company_name+'_income-statement_Annual_As_Originally_Reported' : 'items'} , inplace=True)
balance_sheet.rename(columns={company_name+'_balance-sheet_Annual_As_Originally_Reported' : 'items'} , inplace=True)
cashflow_statement.rename(columns={company_name+'_cash-flow_Annual_As_Originally_Reported' : 'items'} , inplace=True)

In [9]:
basic_eps_filter = income_statement['items'] == 'Basic EPS'
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [10]:
WASO_filter = (income_statement['items'] == 'Basic Weighted Average Shares Outstanding') | (income_statement['items'] == 'Diluted Weighted Average Shares Outstanding') | (income_statement['items'] == 'Basic WASO') | (income_statement['items'] == 'Diluted WASO')
waso_pos = income_statement[WASO_filter].index

In [11]:
income_statement.dropna(how='all', inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.dropna(how='all', inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.dropna(how='all', inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [12]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [13]:
temp_df = temp_df.apply(get_share_price_in)

In [14]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [15]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [16]:
income_statement['items'] = income_statement['items'].apply(lambda x: x.strip())

In [17]:
WASO_filter = (income_statement['items'] == 'Basic Weighted Average Shares Outstanding') | (income_statement['items'] == 'Diluted Weighted Average Shares Outstanding') | (income_statement['items'] == 'Basic WASO') | (income_statement['items'] == 'Diluted WASO')

In [18]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [19]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [31]:
is_indexes = [
                   'Total Revenue' ,
                   'Staff Cost, Cost of Revenue',
                   'Cost of Goods and Services',
                   'Cost of Revenue',
                   'Gross Profit',
                   'Selling, General and Administrative Expenses',
                   'Depreciation, Amortization and Depletion',
                   'Other Income/Expense, Operating',
                   'Provisions',
                   'Operating Income/Expenses',
                   'Total Operating Profit/Loss',
                   'Total Net Finance Income/Expense',
                   'Net Investment Income',
                   'Other Income/Expense, Non-Operating',
                   'Non-Operating Income/Expense, Total',
                   'Pretax Income',
                   'Provision for Income Tax'
           ]

In [42]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns[1:],index=is_indexes)
vertical_analysis_is

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
Total Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Staff Cost, Cost of Revenue",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Goods and Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Profit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Selling, General and Administrative Expenses",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Depreciation, Amortization and Depletion",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Other Income/Expense, Operating",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Provisions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Operating Income/Expenses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
filt = (income_statement['items'] == 'Total Revenue') | (income_statement['items'] == 'Operating Income/Expenses') 
income_statement[filt]

,items,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
1,Total Revenue,629894.8,818093.6,946484.1,1086462.1,1179660.0,1231040.0,1464630.0,1569490.0,1641770.0,1917540.0,2168870.0
7,Operating Income/Expenses,-192894.2,-250606.1,-294143.0,-336086.2,-230880.0,-233000.0,-284970.0,-305120.0,-346200.0,-428990.0,-492340.0
